# Homogeneous Catalysis
[reference](https://pubs.rsc.org/en/content/articlehtml/2017/sc/c7sc03628k)

In [1]:
import sqlite3
import numpy as np

In [2]:
## INITIAL STATE DATABSE ##

create_initial_state_table = """
    CREATE TABLE initial_state (
            species_id             INTEGER NOT NULL PRIMARY KEY,
            count                  INTEGER NOT NULL
    );
"""

create_trajectories_table = """
    CREATE TABLE trajectories (
            seed                INTEGER NOT NULL,
            step                INTEGER NOT NULL,
            time                REAL NOT NULL,
            reaction_id         INTEGER NOT NULL
    );
"""

create_factors_table = """
    CREATE TABLE factors (
            factor_zero      REAL NOT NULL,
            factor_two       REAL NOT NULL,
            factor_duplicate REAL NOT NULL
    );
"""

create_interrupt_state = """
    CREATE TABLE interrupt_state (
            seed                    INTEGER NOT NULL,
            species_id              INTEGER NOT NULL,
            count                   INTEGER NOT NULL
            
    );
"""

create_interrupt_cutoff = """
    CREATE TABLE interrupt_cutoff (
            seed                    INTEGER NOT NULL,
            step                    INTEGER NOT NULL,
            time                    INTEGER NOT NULL
            
    );
"""

In [3]:
def create_state_sqlite(path, stmts, species, initial_state,
                        factor_zero = 1, factor_two = 1, 
                        factor_duplicate = 0.5):
    
    '''Create state sqlite which consists of tables: 
    initial state, trajectories, factors, interrupt_state 
    and interrupt_cutoff'''
    
    rn_con = sqlite3.connect(path)
    rn_cur = rn_con.cursor()
    
    for stmt in stmts:    
        rn_cur.execute(stmt)
    rn_con.commit()
    
    insert_into_factors = "INSERT INTO factors VALUES (?,?,?)"

    rn_cur.execute(
        insert_into_factors,
        (factor_zero, factor_two, factor_duplicate)
    )

    for i in range(len(species)):
        rn_cur.execute(
            "INSERT INTO initial_state VALUES (?,?)",
            (i, initial_state.get(i,0)))

    rn_con.commit()
    rn_con.close()

In [15]:
## MAPPING OF SPECIES
species_thermo = {'3': {'g': 7.5}, '3_p': {'g': -0.7}, 'R': {'g': 0.0}, 
                  '2': {'g': -11.7}, 'I_1':{'g': -3.6}, 'I_2':{'g': -11.7},
                  'I_3':{'g': -21.4}, '27_p':{'g': -5.6}, 'P_side':{'g': -23.8},
                  '14':{'g': -16.2}, '121':{'g': -15.0}, 'I_4':{'g': -17.3},
                  '122':{'g': 0.8}, 'I_4-3':{'g': -13.9}, '20':{'g': 13.2}, 
                  '109':{'g': 19.2}, 'I_5p':{'g': -11.7}, 'I_5':{'g': -4.6}, 
                  'P_p':{'g': -16.9}, 'P':{'g': -21.0},'P_pp':{'g': -7.9}, 
                  'I_5-2':{'g': 0.0}, 'I_5-2p':{'g': -6.6}, 'I_4-2':{'g': -18.7}, 
                  '5':{'g': 26.4}, '23':{'g': 22.1}, '7':{'g': 5.8}, '24':{'g': 20.7}, 
                  '99':{'g': 36.9}, 'foo':{'g': 0.0}, 'foo_1':{'g': 0.0}}

In [22]:
mapping = {m: i for i, m in enumerate(species_thermo, 0)}

In [ ]:
# Create state sqlite file 
initial_state = {0: 400000000000, 1: 400000000000}

stmts = [create_initial_state_table, create_trajectories_table, 
         create_factors_table, create_interrupt_cutoff, 
         create_interrupt_state]

create_state_sqlite("state.sqlite", stmts, species, initial_state)

In [10]:
## CREATE REACTION NETWORK DATABASE ##
create_metadata_table = """
    CREATE TABLE metadata (
            number_of_species   INTEGER NOT NULL,
            number_of_reactions INTEGER NOT NULL
    );
"""

insert_metadata = """
    INSERT INTO metadata VALUES (?, ?)
"""

# it is important that reaction_id is the primary key
# otherwise the network loader will be extremely slow.

create_reactions_table = """
    CREATE TABLE reactions (
            reaction_id                     INTEGER NOT NULL PRIMARY KEY,
            number_of_reactants             INTEGER NOT NULL,
            number_of_products              INTEGER NOT NULL,
            reactant_1                      INTEGER NOT NULL,
            reactant_2                      INTEGER NOT NULL,
            product_1                       INTEGER NOT NULL,
            product_2                       INTEGER NOT NULL,
            rate                            REAL NOT NULL
    );
"""


insert_reaction = """
    INSERT INTO reactions VALUES (?, ?, ?, ?, ?, ?, ?, ?)
"""

In [11]:
def create_reaction_sqlite(path, stmts, num_species, reactions):
    ''' Create an empty sqlite, name, with a table for the metadata and 
    reactions'''
    
    rn_con = sqlite3.connect(path)
    rn_cur = rn_con.cursor()
    
    for stmt in stmts:    
        rn_cur.execute(stmt)
    rn_con.commit()
    
    rn_cur.execute(
        insert_metadata,
        (num_species, np.shape(reactions)[0]))

    rn_con.commit()
    
    for reaction in reactions:
        rn_cur.execute(insert_reaction, tuple(reaction))
    rn_con.commit()
    
    rn_con.close()

In [14]:
reactions = [
   [0, 1, 1, '3', -1, '3_p', -1],
   [1, 2, 1, '3_p', '2', 'R', -1],
   [2, 1, 1, 'I_1', -1, 'R', -1],
   [3, 1, 1, 'P_side', -1, '27_p', -1],
   [4, 1, 1, 'I_3', -1, 'I_4', -1],
   [5, 1, 1, 'I_4', -1, 'I_5p', -1],
   [6, 1, 1, 'I_5p', -1, 'I_5', -1],
   [7, 1, 1, 'I_5', -1, 'P_p', -1],
   [8, 1, 1, '14', -1, '121', -1],
   [9, 1, 1, '20', -1, 'I_4-3', -1], 
   [10, 1, 1, '109', -1, '20', -1],
   [11, 1, 1, 'P_p', -1, 'P', -1],
   [12, 1, 1, 'P', -1, 'P_pp', -1],
   [13, 1, 1, 'P_pp', -1, 'I_5-2', -1],
   [14, 1, 1, 'I_5-2', -1, 'I_5-2p', -1],
   [15, 1, 1, 'I_5-2p', -1, 'I_4-2', -1],

   # reactions with three reactants, 
   # use foo as intermediate 
   [16, 2, 1, 'I_1', '27_p', 'foo', -1],
   [17, 2, 1, 'foo', 'I_3', 'I_2', -1],
   [18, 2, 1, '121', '122', 'foo_1', -1],
   [19, 2, 1, 'foo_1', 'I_4-3', 'I_4', -1] 
]

In [ ]:
## Calculate rates with Eyring equation 

# Room temperature (25 C) in Kelvin
ROOM_TEMP = 298.15

def eyring(dg_barrier, kappa=1.0, temperature=ROOM_TEMP):

    # Boltzmann constant in eV / K
    KB = 8.617333262 * 10 ** -5

    # Planck constant in eV * s
    PLANCK = 4.135667696 * 10 ** -15

    if dg_barrier <= 0:
        return kappa * KB * temperature / PLANCK
    else:
        return kappa * KB * temperature / PLANCK * np.exp(-dg_barrier / (KB * temperature))
